TAHAP 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas requests beautifulsoup4 pdfminer.six lxml > /dev/null 2>&1

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import glob
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from collections import Counter
import json
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
# PERSIAPAN FOLDER DI GOOGLE DRIVE
raw_dir = '/content/drive/MyDrive/ProyekA/data/raw'
logs_dir = '/content/drive/MyDrive/ProyekA/logs'
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)

# FUNGSI HELPER
def create_path(folder_name):
    path = os.path.join(os.getcwd(), folder_name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path

def open_page(link):
    count = 0
    while count < 3:
        try:
            return BeautifulSoup(requests.get(link).text, "lxml")
        except:
            count += 1
            time.sleep(5)

def get_detail(soup, keyword):
    try:
        text = (
            soup.find(lambda tag: tag.name == "td" and keyword in tag.text)
            .find_next()
            .get_text()
            .strip()
        )
        return text
    except:
        return ""

def get_pdf(url, path_pdf):
    try:
        file = urllib.request.urlopen(url)
        file_name = os.path.basename(url)
        file_content = file.read()
        with open(f"{path_pdf}/{file_name}", "wb") as out_file:
            out_file.write(file_content)
        return io.BytesIO(file_content), file_name
    except:
        return None, None

def clean_text(text):
    text = text.replace("M a h ka m a h A g u n g R e p u blik In d o n esia\n", "")
    text = text.replace("Disclaimer\n", "")
    text = text.replace(
        "Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas\n",
        "",
    )
    text = text.replace(
        "pelaksanaan fungsi peradilan. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu.\n",
        "",
    )
    text = text.replace(
        "Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui :\n",
        "",
    )
    text = text.replace(
        "Email : kepaniteraan@mahkamahagung.go.id    Telp : 021-384 3348 (ext.318)\n",
        "",
    )
    return text

# FUNGSI UTAMA UNTUK EKSTRAKSI DATA
def extract_data(link, keyword_url, path_output, path_pdf, today):
    soup = open_page(link)
    table = soup.find("table", {"class": "table"})
    judul = table.find("h2").text if table.find("h2") else ""

    nomor = get_detail(table, "Nomor")
    tingkat_proses = get_detail(table, "Tingkat Proses")
    klasifikasi = get_detail(table, "Klasifikasi")
    kata_kunci = get_detail(table, "Kata Kunci")
    tahun = get_detail(table, "Tahun")
    tanggal_register = get_detail(table, "Tanggal Register")
    lembaga_peradilan = get_detail(table, "Lembaga Peradilan")
    jenis_lembaga_peradilan = get_detail(table, "Jenis Lembaga Peradilan")
    hakim_ketua = get_detail(table, "Hakim Ketua")
    hakim_anggota = get_detail(table, "Hakim Anggota")
    panitera = get_detail(table, "Panitera")
    amar = get_detail(table, "Amar")
    amar_lainnya = get_detail(table, "Amar Lainnya")
    catatan_amar = get_detail(table, "Catatan Amar")
    tanggal_musyawarah = get_detail(table, "Tanggal Musyawarah")
    tanggal_dibacakan = get_detail(table, "Tanggal Dibacakan")
    kaidah = get_detail(table, "Kaidah")
    status = get_detail(table, "Status")
    abstrak = get_detail(table, "Abstrak")

    try:
        link_pdf = soup.find("a", href=re.compile(r"/pdf/"))["href"]
        file_pdf, file_name_pdf = get_pdf(link_pdf, path_pdf)
        text_pdf = extract_text(file_pdf)
        text_pdf = clean_text(text_pdf)
    except:
        link_pdf = ""
        text_pdf = ""
        file_name_pdf = ""

    #Simpan .txt RAW
    raw_filename = re.sub(r'[^a-zA-Z0-9]', '_', judul)[:100] or 'no_title'
    raw_file_path = os.path.join(raw_dir, f"{raw_filename}.txt")
    raw_text = judul + '\n' + (text_pdf or "")
    with open(raw_file_path, 'w', encoding='utf-8') as raw_file:
        raw_file.write(raw_text)

    #LOG CLEANING & PANJANG TEKS
    panjang_asli = len(raw_text)
    panjang_clean = len(clean_text(raw_text))
    status_validasi = "VALID" if panjang_clean >= 0.8 * panjang_asli else "INVALID"

    with open(f"{logs_dir}/cleaning.log", "a") as log_file:
        log_file.write(
            f"{raw_filename}.txt | panjang_asli: {panjang_asli}, panjang_clean: {panjang_clean}, status: {status_validasi}\n"
        )

    #SIMPAN KE CSV
    data = [
        judul,
        nomor,
        tingkat_proses,
        klasifikasi,
        kata_kunci,
        tahun,
        tanggal_register,
        lembaga_peradilan,
        jenis_lembaga_peradilan,
        hakim_ketua,
        hakim_anggota,
        panitera,
        amar,
        amar_lainnya,
        catatan_amar,
        tanggal_musyawarah,
        tanggal_dibacakan,
        kaidah,
        status,
        abstrak,
        link,
        link_pdf,
        file_name_pdf,
        text_pdf,
    ]
    result = pd.DataFrame(
        [data],
        columns=[
            "judul",
            "nomor",
            "tingkat_proses",
            "klasifikasi",
            "kata_kunci",
            "tahun",
            "tanggal_register",
            "lembaga_peradilan",
            "jenis_lembaga_peradilan",
            "hakim_ketua",
            "hakim_anggota",
            "panitera",
            "amar",
            "amar_lainnya",
            "catatan_amar",
            "tanggal_musyawarah",
            "tanggal_dibacakan",
            "kaidah",
            "status",
            "abstrak",
            "link",
            "link_pdf",
            "file_name_pdf",
            "text_pdf",
        ],
    )

    keyword_url = keyword_url.replace("/", " ")
    if keyword_url.startswith("https"):
        keyword_url = ""
    destination = f"{path_output}/putusan_ma_{keyword_url}_{today}.csv"

    if not os.path.isfile(destination):
        result.to_csv(destination, header=True, index=False)
    else:
        result.to_csv(destination, mode="a", header=False, index=False)

#MAIN FUNCTION
def run_process(keyword_url, page, sort_date, path_output, path_pdf, today):
    if keyword_url.startswith("https"):
        link = f"{keyword_url}&page={page}"
    else:
        link = f"https://putusan3.mahkamahagung.go.id/search.html?q={keyword_url}&page={page}"
    if sort_date:
        link = f"{link}&obf=TANGGAL_PUTUS&obm=desc"

    soup = open_page(link)
    links = soup.find_all("a", {"href": re.compile("/direktori/putusan")})
    for link in links:
        extract_data(link["href"], keyword_url, path_output, path_pdf, today)

def run_scraper(keyword=None, url=None, sort_date=True, download_pdf=True):
    if not keyword and not url:
        print("Please provide a keyword or URL.")
        return

    path_output = '/content/drive/MyDrive/ProyekA/CSV'
    path_pdf = '/content/drive/MyDrive/ProyekA/PDF'
    today = date.today().strftime("%Y-%m-%d")

    link = f"https://putusan3.mahkamahagung.go.id/search.html?q={keyword}&page=1"
    if url:
        link = url

    soup = open_page(link)
    last_page = int(soup.find_all("a", {"class": "page-link"})[-1].get("data-ci-pagination-page"))

    if url:
        print(f"Scraping with url: {url} - {20 * last_page} data - {last_page} page")
    else:
        print(f"Scraping with keyword: {keyword} - {20 * last_page} data - {last_page} page")

    keyword_url = url if url else keyword
    futures = []
    with ThreadPoolExecutor(max_workers=4) as executor:
        for page in range(last_page):
            futures.append(
                executor.submit(run_process, keyword_url, page + 1, sort_date, path_output, path_pdf, today)
            )
    wait(futures)


In [ ]:
# Download Pidana PN MAKASSAR
run_scraper(url="https://putusan3.mahkamahagung.go.id/search.html?q=Sulawesi&jenis_doc=putusan&tp=0&t_reg=2024&court=099422PN142&cat=bfa5809fc342e6a6ef5d3d9de5ec7075&jd=AMAR_LAINNYA")

Scraping with url: https://putusan3.mahkamahagung.go.id/search.html?q=Sulawesi&jenis_doc=putusan&tp=0&t_reg=2024&court=099422PN142&cat=bfa5809fc342e6a6ef5d3d9de5ec7075&jd=AMAR_LAINNYA - 40 data - 2 page
